In [63]:
import os
import pandas as pd

In [38]:
# pip install python-dotenv
from dotenv import load_dotenv

%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [3]:
# fetch dataset from ADX
# pip install azure-kusto-data
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.helpers import dataframe_from_result_table
import pandas as pd

AAD_TENANT_ID = os.environ['AAD_TENANT_ID']
KUSTO_CLUSTER = os.environ['KUSTO_CLUSTER']
KUSTO_DATABASE = os.environ['KUSTO_DATABASE']

In [4]:
# Interactive login
KCSB = KustoConnectionStringBuilder.with_aad_device_authentication(KUSTO_CLUSTER)
KCSB.authority_id = AAD_TENANT_ID


In [42]:
# Service principal login
# https://github.com/Azure/azure-kusto-python/tree/master/azure-kusto-data
# https://github.com/Azure/azure-kusto-python/blob/master/azure-kusto-data/tests/sample.py

# client_id = os.environ['CLIENT_ID']
# client_secret = os.environ['CLIENT_SECRET']
# authority_id = os.environ['AAD_TENANT_ID']
# KCSB = KustoConnectionStringBuilder.with_aad_application_key_authentication(KUSTO_CLUSTER, client_id, client_secret, authority_id)

In [53]:
%%time
KUSTO_CLIENT = KustoClient(KCSB)
KUSTO_QUERY = "malware_table_10000"
# KUSTO_QUERY = "set notruncation; malware_table_1000000"

RESPONSE = KUSTO_CLIENT.execute(KUSTO_DATABASE, KUSTO_QUERY)

CPU times: user 48.2 s, sys: 8.77 s, total: 57 s
Wall time: 2min 50s


In [54]:
%%time
# convert to data frame or native test
df = dataframe_from_result_table(RESPONSE.primary_results[0])
df.head()

CPU times: user 24 s, sys: 2.6 s, total: 26.6 s
Wall time: 26.5 s


,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,...,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,53447.0,...,57858.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,0,7.0,0,NaN,53447.0,...,52682.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,...,20050.0,0,NaN,0.0,0,0,0.0,0.0,3.0,1
4,000014a5f00daa18e76b81417eeb99fc,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1379.0,0,7.0,0,NaN,53447.0,...,19844.0,0,0.0,0.0,0,0,0.0,0.0,1.0,1


In [58]:
len(df)

999999

In [55]:
# Save dataframe to CSV
df.to_csv(r'data/adx-df.csv', index = False, header=True)
# df.to_csv(r'data/adx-df-1000000.csv', index = False, header=True)

In [57]:
df.memory_usage(index=True).sum()
# 663,999,464

663999464